In [1]:
# move to project root
%cd ..
%pwd

/Users/heste/workspace/soccernet/sn-script


/Users/heste/workspace/soccernet/sn-script/.venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/Users/heste/workspace/soccernet/sn-script'

In [34]:
import os

import numpy as np
import pandas as pd

label_template = "commentary_dataset/gpt-3.5-turbo-1106_500game_{half_nunber}_llm_annotation_{split}.csv"

split = "train"
label_df_half1 = pd.read_csv(
    os.path.join(label_template.format(half_nunber=1, split=split))
)
label_df_half2 = pd.read_csv(
    os.path.join(label_template.format(half_nunber=2, split=split))
)

In [8]:
label_df_half1.head()

,id,game,start,end,text,付加的情報か,大分類,小分類,備考,split,target_frameid,target_label
0,0,england_epl/2014-2015/2015-02-21 - 18-00 Chels...,00:00,00:03,"Arrancó ya el duelo, maneja la pelota el Barley.",0.0,NaN,NaN,This comment does not contain any supplementar...,train,0,1.0
1,1,england_epl/2014-2015/2015-02-21 - 18-00 Chels...,00:00,00:09,Hay que decir que también se midieron en el pr...,1.0,NaN,NaN,This comment contains historical match data.,train,0,2.0
2,2,england_epl/2014-2015/2015-02-21 - 18-00 Chels...,00:08,00:11,El Barley se adelantó.,0.0,NaN,NaN,This comment refers to a specific event in the...,train,8,1.0
3,3,england_epl/2014-2015/2015-02-21 - 18-00 Chels...,00:09,00:18,Hay que recordar que se adelantó en ese partid...,1.0,NaN,NaN,This comment contains historical match statist...,train,9,2.0
4,4,england_epl/2014-2015/2015-02-21 - 18-00 Chels...,00:17,00:19,"Exactamente, justo lo que iba a decir.",0.0,NaN,NaN,This comment does not contain any supplementar...,train,17,1.0


In [3]:
games_half1 = label_df_half1["game"].unique().tolist()
games_half2 = label_df_half2["game"].unique().tolist()

In [5]:
comment_counts_half1 = { game: len(ids) for game, ids in label_df_half1.groupby('game').groups.items()}

In [6]:
mean_comment_counts_half1 = sum(comment_counts_half1.values()) / len(comment_counts_half1)
print(f"{mean_comment_counts_half1=}")

mean_comment_counts_half1=487.8929889298893


In [7]:
average_comment_interval_half1 = round(45 * 60 / mean_comment_counts_half1)
print(f"{average_comment_interval_half1=}")

average_comment_interval_half1=6


In [10]:
comment_addional_info_counts_half1 = {
    game: len(ids)
    for game, ids in label_df_half1[label_df_half1["付加的情報か"] == 1].groupby('game').groups.items()
}
mean_comment_addional_info_counts_half1 = sum(comment_addional_info_counts_half1.values()) / len(comment_addional_info_counts_half1)
print(f"{mean_comment_addional_info_counts_half1=}")
average_comment_addional_info_interval_half1 = round(45 * 60 / mean_comment_addional_info_counts_half1)
print(f"{average_comment_addional_info_interval_half1=}")

mean_comment_addional_info_counts_half1=70.79377431906615
average_comment_addional_info_interval_half1=38


In [6]:
len(games_half1), len(games_half2), len(set(games_half1) & set(games_half2))

(271, 275, 270)

In [7]:
from SoccerNet.Downloader import getListGames
games = getListGames("train", "caption")

In [8]:
len(games)

281

In [10]:
len(label_df_half1)

132219

In [11]:
label_df_half1 = label_df_half1.dropna(subset=["target_label"])

In [12]:
len(label_df_half1)

131724

In [8]:
# number of comments per game
comments_per_game = label_df_half1.groupby("game").size()

game
england_epl/2014-2015/2015-02-21 - 18-00 Chelsea 1 - 1 Burnley               751
england_epl/2014-2015/2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal        400
england_epl/2014-2015/2015-02-21 - 18-00 Swansea 2 - 1 Manchester United     352
england_epl/2014-2015/2015-02-22 - 19-15 Southampton 0 - 2 Liverpool         650
england_epl/2015-2016/2015-08-08 - 19-30 Chelsea 2 - 2 Swansea               746
                                                                            ... 
spain_laliga/2016-2017/2017-05-06 - 21-45 Granada CF 0 - 4 Real Madrid       892
spain_laliga/2016-2017/2017-05-14 - 21-00 Real Madrid 4 - 1 Sevilla          677
spain_laliga/2016-2017/2017-05-17 - 22-00 Celta Vigo 1 - 4 Real Madrid      1187
spain_laliga/2016-2017/2017-05-21 - 21-00 Barcelona 4 - 2 Eibar              565
spain_laliga/2016-2017/2017-05-21 - 21-00 Malaga 0 - 2 Real Madrid           677
Length: 271, dtype: int64

In [20]:
gdf = label_df_half1.groupby("game")
gdf.get_group("game")

KeyError: 'game'

In [33]:
label_df_half2["start"]

0         00:00
1         00:00
2         00:07
3         00:13
4         00:20
          ...  
131056    44:50
131057    44:52
131058    44:54
131059    44:55
131060    44:57
Name: start, Length: 131061, dtype: object

In [40]:
def gametime_to_seconds(gametime):
    # TODO csv_utils.pyからのインポートに変更
    m, s = gametime.split(":")
    return int(m) * 60 + int(s)

def calc_average_interval_gap(df: pd.DataFrame) -> dict[str, int]:
    """
    直前の行のstartと、今の行のstartの差分の平均を計算する
    startは hh:mm 形式の文字列であることを前提とする
    すでにstartでソートされていることを前提とする
    差分がnanとなる値,ゲームの最初のコメントは無視する

    出力
    合計: ラベルによらない総計
    映像の説明: 今の行のラベルが「付加的情報を含まない」のinterval平均
    付加的情報: 今の行のラベルが「付加的情報を含む」のinterval平均
    """
    # 非破壊的な操作を行うためにコピーを作成
    df = df.copy()

    df["start"] = df["start"].apply(gametime_to_seconds)
    df["start_prev"] = df["start"].shift(1)
    df["interval"] = df["start"] - df["start_prev"]

    conditions = (df["interval"] < 0) | (df["interval"].isna()) | (df["game"] != df["game"].shift(1))
    df.loc[conditions, "interval"] = np.nan


    result = df["interval"].mean()
    result_sitinvid = df.loc[df["付加的情報か"] == 0, "interval"].mean()
    result_addinfo = df.loc[df["付加的情報か"] == 1, "interval"].mean()

    result_dict = {
        "合計": result,
        "映像の説明": result_sitinvid,
        "付加的情報": result_addinfo
    }

    return result_dict

print(f"{calc_average_interval_gap(label_df_half1)=}")
print(f"{calc_average_interval_gap(label_df_half2)=}")

calc_average_interval_gap(label_df_half1)={'合計': 4.916800557795495, '映像の説明': 4.678105527017246, '付加的情報': 6.340345853067518}
calc_average_interval_gap(label_df_half2)={'合計': 4.98979248543422, '映像の説明': 4.709697512018809, '付加的情報': 6.968126266658478}
